## Fix IPUMS conflicting shapes

in IPUMS shapes there are two kind of errors:
1) Conflicting territories, that are reported as being part of two different countries,
2) Conflicting geometries, where, for some reason, two geometries of a single country overlap and missasing where the data comes from.

For the first point, we use the WB official boundaries. For the second, we remove the wrong duplicate (remove PR from USA, remove small polygon for Laos).   

In [32]:
import pandas as pd
import geopandas as gpd

wb = gpd.read_file(r"D:\World Bank\CLIENT v2\Data\Data_raw\world_bank_adm2\world_bank_adm2.shp")
israel = gpd.read_file(r"D:\World Bank\CLIENT v2\Data\Data_raw\IPUMS Fixed\geo2_il1972_1995.shp")

In [97]:
## ISRAEL
israel_ipums = gpd.read_file(r"D:\World Bank\CLIENT v2\Data\Data_raw\IPUMS Fixed\geo2_il1972_1995.shp")
israel_ipums["geometry"] = israel_ipums.centroid
israel_ipums = israel_ipums.set_crs(epsg=4326)

israel_WB = wb[wb.ADM0_NAME == "Israel"][["ADM1_CODE", "ADM2_CODE", "geometry"]]

# Genera csv con el fix
israel_WB.sjoin(israel_ipums)[["GEOLEVEL2", "ADM1_CODE"]].set_index("GEOLEVEL2").to_csv(r"D:\World Bank\CLIENT v2\Data\Data_proc\fix_israel_geo2_adm1.csv")

# Muestra los datos
m = israel_WB.explore()
israel_ipums.explore(m=m, color="red") 

In [94]:
israel_ipums.explore()

In [100]:
import folium

## PALESTINE
palestine_ipums = gpd.read_file(r"D:\World Bank\CLIENT v2\Data\Data_raw\IPUMS Fixed\geo1_ps1997_2017.shp")
pal_ipums = palestine_ipums.copy()
pal_ipums["geometry"] = pal_ipums.centroid
pal_ipums = pal_ipums.set_crs(epsg=4326)

pal_WB = wb[wb.ADM0_NAME == "West Bank and Gaza"][["ADM1_CODE", "ADM2_CODE", "geometry"]]

# Genera csv con el fix
pal_WB.sjoin(pal_ipums)[["GEOLEVEL1", "ADM1_CODE"]].set_index("GEOLEVEL1").to_csv(r"D:\World Bank\CLIENT v2\Data\Data_proc\fix_palestine_geo1_adm1.csv")

# Muestra los datos
m = pal_WB.explore()
# palestine_ipums.explore(m=m, color="orange")
pal_ipums.explore(m=m, color="red")

folium.LayerControl().add_to(m)  # use folium to add layer control
m


In [29]:
# Visualize overlaps
wb = gpd.read_file(r"D:\World Bank\CLIENT v2\Data\Data_raw\world_bank_adm2\world_bank_adm2.shp")
wb.geometry = wb.buffer(-0.0001)
# Perform a spatial join between the GeoDataFrame and itself to find overlaps
overlaps = gpd.sjoin(wb, wb, how='inner', predicate='intersects')

# Filter out self-overlaps
overlaps = overlaps[overlaps.OBJECTID_left != overlaps.OBJECTID_right]

# Display the overlapping rows
overlaps


C:\Users\ofici\AppData\Local\Temp\ipykernel_14816\118252492.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  wb.geometry = wb.buffer(-0.0001)


,OBJECTID_left,ADM2_CODE_left,ADM2_NAME_left,STR2_YEAR_left,EXP2_YEAR_left,ADM1_CODE_left,ADM1_NAME_left,STATUS_left,DISP_AREA_left,ADM0_CODE_left,ADM0_NAME_left,Shape_Leng_left,Shape_Le_1_left,Shape_Area_left,geometry,index_right,OBJECTID_right,ADM2_CODE_right,ADM2_NAME_right,STR2_YEAR_right,EXP2_YEAR_right,ADM1_CODE_right,ADM1_NAME_right,STATUS_right,DISP_AREA_right,ADM0_CODE_right,ADM0_NAME_right,Shape_Leng_right,Shape_Le_1_right,Shape_Area_right
10393,10394.0,12552,Greater Vancouver,1000,3000,826,British Columbia / Colombie-Britannique,Member State,NO,46,Canada,6.948402,6.948402,0.378433,"MULTIPOLYGON (((-123.29341 49.37230, -123.2934...",13619,13620.0,31666,Whatcom,1000,3000,3261,Washington,Member State,NO,259,United States of America,7.060766,7.060766,0.686249
10484,10485.0,12649,Essex,1000,3000,833,Ontario,Member State,NO,46,Canada,4.409607,4.409607,0.203409,"MULTIPOLYGON (((-83.11673 42.06873, -83.11643 ...",11949,11950.0,29990,Wayne,1000,3000,3236,Michigan,Member State,NO,259,United States of America,3.322123,3.322123,0.174413
10498,10499.0,12663,Leeds and Grenville,1000,3000,833,Ontario,Member State,NO,46,Canada,7.329470,7.329470,0.409942,"MULTIPOLYGON (((-75.36734 44.79660, -75.36728 ...",12486,12487.0,30527,Jefferson,1000,3000,3246,New York,Member State,NO,259,United States of America,7.814186,7.814186,0.373373
11949,11950.0,29990,Wayne,1000,3000,3236,Michigan,Member State,NO,259,United States of America,3.322123,3.322123,0.174413,"MULTIPOLYGON (((-83.28940 42.09050, -83.28937 ...",10484,10485.0,12649,Essex,1000,3000,833,Ontario,Member State,NO,46,Canada,4.409607,4.409607,0.203409
12486,12487.0,30527,Jefferson,1000,3000,3246,New York,Member State,NO,259,United States of America,7.814186,7.814186,0.373373,"MULTIPOLYGON (((-76.33468 44.12997, -76.33467 ...",10498,10499.0,12663,Leeds and Grenville,1000,3000,833,Ontario,Member State,NO,46,Canada,7.329470,7.329470,0.409942
13619,13620.0,31666,Whatcom,1000,3000,3261,Washington,Member State,NO,259,United States of America,7.060766,7.060766,0.686249,"MULTIPOLYGON (((-122.78941 48.97498, -122.7894...",10393,10394.0,12552,Greater Vancouver,1000,3000,826,British Columbia / Colombie-Britannique,Member State,NO,46,Canada,6.948402,6.948402,0.378433
19454,19455.0,12959,Antartica Chilena,1000,3000,891,Magallanes y Antartica chilena,Member State,NO,51,Chile,136.370785,136.370785,2.222938,"MULTIPOLYGON (((-65.55304 -54.64822, -65.55344...",22119,22120.0,4872,Ushuaia,1000,3000,451,Tierra Del Fuego,Member State,NO,12,Argentina,17.639814,17.639814,1.246633
22119,22120.0,4872,Ushuaia,1000,3000,451,Tierra Del Fuego,Member State,NO,12,Argentina,17.639814,17.639814,1.246633,"MULTIPOLYGON (((-64.75195 -54.83499, -64.75333...",19454,19455.0,12959,Antartica Chilena,1000,3000,891,Magallanes y Antartica chilena,Member State,NO,51,Chile,136.370785,136.370785,2.222938
